In [90]:
import numpy as np
import time
import random

# Homework 2

### Problem 3

In [91]:
def gd(x1, df, lr, n, time_limit=None):
    t = time.time()
    points = [x1]
    for _ in range(n):
        x = points[-1]
        points.append(x - lr*df(x))
        if time_limit is not None:
            if time.time() - t > time_limit:
                print('Time limit reached - gd')
                break
    return points

def polyak_gd(x1, df, lr, mu, n, time_limit=None):
    t = time.time()
    points = [x1, x1]
    for _ in range(n):
        x = points[-1]
        points.append(x - lr*df(x) + mu*(x - points[-2]))
        if time_limit is not None:
            if time.time() - t > time_limit:
                print('Time limit reached - polyak')
                break
    return points

def nesterov_gd(x1, df, lr, mu, n, time_limit=None):
    t = time.time()
    points = [x1, x1]
    for _ in range(n):
        x = points[-1]
        points.append(x - lr*df(x + mu*(x - points[-2])) + mu*(x - points[-2]))
        if time_limit is not None:
            if time.time() - t > time_limit:
                print('Time limit reached - nesterov')
                break
    return points

def adagrad(x1, df, lr, n, time_limit=None):
    t = time.time()
    points = [x1]
    d = np.ones([x1.shape[0], 1]) * 1e-10
    for _ in range(n):
        x = points[-1]
        d += df(x)**2
        D = np.diag(d**(-0.5))
        points.append(x - lr*D*df(x))
        if time_limit is not None:
            if time.time() - t > time_limit:
                print('Time limit reached - adagrad')
                break
    return points

In [92]:
gd(10, lambda x: 2*x, 0.1, 100)[-1]

2.0370359763344878e-09

In [93]:
polyak_gd(10, lambda x: 2*x, 0.1, 0.1, 100)[-1]

4.760651453586529e-11

In [94]:
nesterov_gd(10, lambda x: 2*x, 0.1, 0.1, 100)[-1]

1.1460793354650304e-10

In [95]:
adagrad(np.array([[2], [1]]), lambda x: np.array([2*x[0], 2*x[1]]), 0.1, 1000)[-1]

array([[1.88011605e-04],
       [9.40058025e-05]])

### Problem 4

In [96]:
def newton(x1, df, H, n, time_limit=None):
    t = time.time()
    points = [x1]
    for _ in range(n):
        p = points[-1]
        points.append(p - np.linalg.inv(H(p)).dot(df(p)))
        if time_limit is not None:
            if time.time() - t > time_limit:
                print('Time limit reached - newton')
                break
    return points


def bfgs(x1, df, n, time_limit=None):
    t = time.time()
    B = [np.identity(x1.shape[0])]
    points = [x1,
              x1 - B[0].dot(df(x1))]
    for _ in range(n-1):
        p1 = points[-1]
        p2 = points[-2]
        gamma = df(p1) - df(p2)
        delta = p1 - p2
        if delta.T.dot(gamma) == 0:
            print('bfgs: stopped because of invalid values')
            return points
        Bk = B[-1] - (delta.dot(gamma.T.dot(B[-1])) + B[-1].dot(gamma).dot(delta.T)) / (delta.T.dot(gamma)) + \
             (1 + (gamma.T.dot(B[-1]).dot(gamma)) / (delta.T.dot(gamma))) * (delta * delta.T) / (delta.T.dot(gamma))
        
        B.append(Bk)
        points.append(p1 - Bk.dot(df(p1)))
        if time_limit is not None:
            if time.time() - t > time_limit:
                print('Time limit reached - bfgs')
                break
    return points


### Problem 5

Function f1 has a global minimum at point at x\* = (-1/6, -11/48, 1/6), f1(x\*) = -19/96.

Function f2 has a global minimum at point at x\* = (1, 1, 1), f2(x\*) = 0.

Function f2 has a global minimum at point at x\* = (3, 0.5), f3(x\*) = 0.

In [97]:
def df1(X):
    x, y, z = X[0, 0], X[1, 0], X[2, 0]
    return np.array([[34 * x - 16 * y + 6 * z + 1, -16 * x + 16 * y + 1, 6 * (x + z)]]).T

def H1(X):
    return np.array([[34, -16, 6], [-16, 16, 0], [6, 0, 6]])

x1_11 = np.array([[0, 0, 0]]).T
x1_12 = np.array([[1, 1, 0]]).T

act_min1 = np.array([[-1/6, -11/48, 1/6]]).T

def df2(X):
    x, y, z = X[0, 0], X[1, 0], X[2, 0]
    return np.array([[2*(x-1) - 400*x*(y-2*x**2), 2*(y-1) + 200*(y-x**2) - 400*y*(z-y**2), 200*(z-y**2)]]).T

def H2(X):
    x, y, z = X[0, 0], X[1, 0], X[2, 0]
    return np.array([[-400*(y-x**2) + 800*x**2 + 2, -400*x, 0], [-400*x, -400*(z-y**2) + 800*y**2 + 202, -400*y],
                     [0, -400*y, 200]])

x1_21 = np.array([[1.2, 1.2, 1.2]]).T
x1_22 = np.array([[-1, 1.2, 1.2]]).T

act_min2 = np.array([[1, 1, 1]]).T

def df3(X):
    x, y = X[0, 0], X[1, 0]
    return np.array([[2*(1.5 - x + x*y)*(y-1) + 2*(2.25 - x + x*y**2)*(y**2-1) + 2*(2.625 - x + x*y**3)*(y**3-1),
                      2*(1.5 - x + x*y)*x + 4*(2.25 - x + x*y**2)*(x*y) + 6*(2.625 - x + x*y**3)*(x*y**2)]]).T

def H3(X):
    x, y = X[0, 0], X[1, 0]
    return np.array([[2*(y**6+y**4-2*y**3-y**2-2*y+3), 2*x*(6*y**5+4*y**3 - 6*y**2 - 2*y - 2) + 15.75*y**2 + 9*y + 3],
                     [2*x*(6*y**5 + 4*y**3 - 6*y**2 - 2*y - 2) + 15.75*y**2 + 9*y + 3,
                      2*x*(x*(15*y**4 + 6*y**2 - 6*y - 1) + 6*2.625*y + 4.5*y)]])

x1_31 = np.array([[1, 1]]).T
x1_32 = np.array([[4.5, 4.5]]).T

act_min3 = np.array([[3, 0.5]]).T

In [98]:
def compare_on_fn(grad, hess, start_points, actual_min, lr, mu):
    pt1 = start_points[0]
    pts1 = [gd(pt1, grad, lr, 100), polyak_gd(pt1, grad, lr, mu, 100),
           nesterov_gd(pt1, grad, lr, mu, 100), adagrad(pt1, grad, lr, 100),
           newton(pt1, grad, hess, 100), bfgs(pt1, grad, 100)]
    pt2 = start_points[1]
    pts2 = [gd(pt2, grad, lr, 100), polyak_gd(pt2, grad, lr, mu, 100),
           nesterov_gd(pt2, grad, lr, mu, 100), adagrad(pt2, grad, lr, 100),
           newton(pt2, grad, hess, 100), bfgs(pt2, grad, 100)]
    names = ['GD', 'Polyak', 'Nesterov', 'AdaGrad', 'Newton', 'BFGS']
    for j in range(6):
        steps = np.array([2, 5, 10, 100])
        st1 = steps.copy()
        st2 = steps.copy()
        if names[j] in ['Polyak', 'Nesterov']:  # two starting points saved
            st1 += 1
            st2 += 1
        elif names[j] == 'BFGS':
            st1 = [min(ss, len(pts1[j]) - 1) for ss in st1]
            st2 = [min(ss, len(pts2[j]) - 1) for ss in st2]
        print(f'Distance from actual minimum with method {names[j]}:'
              f'- at starting point {pt1.T}: ')
        for s in range(4):
            dist = np.linalg.norm(pts1[j][st1[s]] - actual_min)
            if names[j] == 'BFGS':
                print(f'    - after {st1[s]} steps: {dist}')
            else:
                print(f'    - after {steps[s]} steps: {dist}')
        print(f'- at starting point {pt2.T}: ')
        for s in range(4):
            dist = np.linalg.norm(pts2[j][st2[s]] - actual_min)
            if names[j] == 'BFGS':
                print(f'    - after {st1[s]} steps: {dist}')
            else:
                print(f'    - after {steps[s]} steps: {dist}')



#### a)

In [99]:
compare_on_fn(df1, H1, [x1_11, x1_12], act_min1, 1e-2, 0.5)

bfgs: stopped because of invalid values
bfgs: stopped because of invalid values
Distance from actual minimum with method GD:- at starting point [[0 0 0]]: 
    - after 2 steps: 0.30556698665486315
    - after 5 steps: 0.27542065817106137
    - after 10 steps: 0.2339933570477582
    - after 100 steps: 0.016388658000053968
- at starting point [[1 1 0]]: 
    - after 2 steps: 1.4815978705438262
    - after 5 steps: 1.258412758444076
    - after 10 steps: 0.9987472242747498
    - after 100 steps: 0.06377134179676562
Distance from actual minimum with method Polyak:- at starting point [[0 0 0]]: 
    - after 2 steps: 0.29974130512827224
    - after 5 steps: 0.24553780474157771
    - after 10 steps: 0.1761915589972354
    - after 100 steps: 0.0005586822297691529
- at starting point [[1 1 0]]: 
    - after 2 steps: 1.4308940969431199
    - after 5 steps: 1.052011154177836
    - after 10 steps: 0.6941834468545387
    - after 100 steps: 0.002173935573733558
Distance from actual minimum with meth

We can see, that Newton method works best already from the second step on. Later it doesn't improve anymore. After 16 steps, BFGS reaches the same distance, and after that, we stopped optimization, because of invalid values. We chose learning rate 0.01, because it is the best for first order methods (with higher one they diverge, with lower they are too slow).

In [100]:
compare_on_fn(df2, H2, [x1_21, x1_22], act_min2, 1e-5, 0.5)


Distance from actual minimum with method GD:- at starting point [[1.2 1.2 1.2]]: 
    - after 2 steps: 0.33723321009653945
    - after 5 steps: 0.32515328779321634
    - after 10 steps: 0.30885732488867595
    - after 100 steps: 0.2824409186767944
- at starting point [[-1.   1.2  1.2]]: 
    - after 2 steps: 2.0133294915561417
    - after 5 steps: 2.003882775340714
    - after 10 steps: 1.9891437488760535
    - after 100 steps: 1.8404417947433258
Distance from actual minimum with method Polyak:- at starting point [[1.2 1.2 1.2]]: 
    - after 2 steps: 0.3349918827710254
    - after 5 steps: 0.31394151400798653
    - after 10 steps: 0.2895622904586029
    - after 100 steps: 0.31126716413940825
- at starting point [[-1.   1.2  1.2]]: 
    - after 2 steps: 2.011673388931375
    - after 5 steps: 1.9942232775800572
    - after 10 steps: 1.9664592480814138
    - after 100 steps: 1.7734360050343498
Distance from actual minimum with method Nesterov:- at starting point [[1.2 1.2 1.2]]: 
    - a

Here BFGS diverges, the reason could be big distance of the starting point from the actual minimum, at least for the second point. Newton method, previosly best method is jumping over minimum. The best ones are normal and Polyak gradient descends, depends on the step we are on.

In [101]:
compare_on_fn(df3, H3, [x1_31, x1_32], act_min3, 1e-6, 0.5)


Distance from actual minimum with method GD:- at starting point [[1 1]]: 
    - after 2 steps: 2.0615393524986976
    - after 5 steps: 2.0615191625244966
    - after 10 steps: 2.0614855138754815
    - after 100 steps: 2.0608801149916327
- at starting point [[4.5 4.5]]: 
    - after 2 steps: 3.8488791340005966
    - after 5 steps: 3.487024466221991
    - after 10 steps: 3.1350931351593223
    - after 100 steps: 1.8941802320781476
Distance from actual minimum with method Polyak:- at starting point [[1 1]]: 
    - after 2 steps: 2.061535987712399
    - after 5 steps: 2.0614985553137104
    - after 10 steps: 2.0614316757539686
    - after 100 steps: 2.060221625120506
- at starting point [[4.5 4.5]]: 
    - after 2 steps: 3.727437724879665
    - after 5 steps: 3.064861130132364
    - after 10 steps: 2.618243300682049
    - after 100 steps: 1.554836009243065
Distance from actual minimum with method Nesterov:- at starting point [[1 1]]: 
    - after 2 steps: 2.0615359875693717
    - after 5 s

C:\Users\ursau\AppData\Local\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\ursau\AppData\Local\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\ursau\AppData\Local\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: overflow encountered in double_scalars
C:\Users\ursau\AppData\Local\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: overflow encountered in double_scalars


Here we needed to set learning rate as small, because otherwise most of our methods divirged (specially at the second starting point). BFGS again divirges. Newton jumpes over the minimum for the second point, on which the best one is Polyak. On the first point, the best are Polyak and Nesterov (slightly better), reaching almost the same result.

In [102]:
def compare_on_fn_time(grad, hess, start_points, actual_min, lr, mu):
    pt1 = start_points[0]
    pt2 = start_points[1]
    names = ['GD', 'Polyak', 'Nesterov', 'AdaGrad', 'Newton', 'BFGS']
    time_limits = [.1, 1, 2]
    
    points = [[[gd(pt, grad, lr, 1000000, t)[-1], polyak_gd(pt, grad, lr, mu, 1000000, t)[-1],
             nesterov_gd(pt, grad, lr, mu, 1000000, t)[-1], adagrad(pt, grad, lr, 1000000, t)[-1],
             newton(pt, grad, hess, 100000, t)[-1], bfgs(pt, grad, 100000, t)[-1]] for t in time_limits] for pt in start_points]
    for j in range(6):
        print(f'Distance from actual minimum with method {names[j]}:'
              f'- at starting point {pt1.T}: ')
        for t in range(3):
            dist = np.linalg.norm(points[0][t][j] - actual_min)
            print(f'    - after {time_limits[t]}s: {dist}')
        print(f'- at starting point {pt2.T}: ')
        for t in range(3):
            dist = np.linalg.norm(points[1][t][j] - actual_min)
            print(f'    - after {time_limits[t]}s: {dist}')

In [103]:
compare_on_fn_time(df1, H1, [x1_11, x1_12], act_min1, 1e-2, 0.5)

Time limit reached - gd
Time limit reached - polyak
Time limit reached - nesterov
Time limit reached - adagrad
Time limit reached - newton
bfgs: stopped because of invalid values
Time limit reached - gd
Time limit reached - polyak
Time limit reached - nesterov
Time limit reached - adagrad
Time limit reached - newton
bfgs: stopped because of invalid values
Time limit reached - gd
Time limit reached - polyak
Time limit reached - nesterov
Time limit reached - adagrad
Time limit reached - newton
bfgs: stopped because of invalid values
Time limit reached - gd
Time limit reached - polyak
Time limit reached - nesterov
Time limit reached - adagrad
Time limit reached - newton
bfgs: stopped because of invalid values
Time limit reached - gd
Time limit reached - polyak
Time limit reached - nesterov
Time limit reached - adagrad
Time limit reached - newton
bfgs: stopped because of invalid values
Time limit reached - gd
Time limit reached - polyak
Time limit reached - nesterov
Time limit reached - ad

The best ones for the first function are also from timing point of view Newton and BFGS, very quickly reaching their optimum, better than all others already in 0.1s.

In [104]:
compare_on_fn_time(df2, H2, [x1_21, x1_22], act_min2, 1e-5, 0.5)

Time limit reached - gd
Time limit reached - polyak
Time limit reached - nesterov
Time limit reached - adagrad
Time limit reached - newton
Time limit reached - bfgs
Time limit reached - gd
Time limit reached - polyak
Time limit reached - nesterov
Time limit reached - adagrad
Time limit reached - newton
Time limit reached - bfgs
Time limit reached - gd
Time limit reached - polyak
Time limit reached - nesterov
Time limit reached - adagrad
Time limit reached - newton
Time limit reached - bfgs
Time limit reached - gd
Time limit reached - polyak
Time limit reached - nesterov
Time limit reached - adagrad
Time limit reached - newton
Time limit reached - bfgs
Time limit reached - gd
Time limit reached - polyak
Time limit reached - nesterov
Time limit reached - adagrad
Time limit reached - newton
Time limit reached - bfgs
Time limit reached - gd
Time limit reached - polyak
Time limit reached - nesterov
Time limit reached - adagrad
Time limit reached - newton
Time limit reached - bfgs
Distance f

For the first starting point, the best results are reached by AdaGrad, again reaching better distance then all others in the first 0.1s. For the second point, gradient descends (normal, polyak and nesterov) work the best.

In [105]:
compare_on_fn_time(df3, H3, [x1_31, x1_32], act_min3, 1e-6, 0.5)

Time limit reached - gd
Time limit reached - polyak
Time limit reached - nesterov
Time limit reached - adagrad
Time limit reached - newton
Time limit reached - bfgs


C:\Users\ursau\AppData\Local\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\ursau\AppData\Local\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in double_scalars


Time limit reached - gd
Time limit reached - polyak
Time limit reached - nesterov
Time limit reached - adagrad
Time limit reached - newton
Time limit reached - bfgs
Time limit reached - gd
Time limit reached - polyak
Time limit reached - nesterov
Time limit reached - adagrad
Time limit reached - newton
Time limit reached - bfgs
Time limit reached - gd
Time limit reached - polyak
Time limit reached - nesterov
Time limit reached - adagrad
Time limit reached - newton


C:\Users\ursau\AppData\Local\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: overflow encountered in double_scalars
C:\Users\ursau\AppData\Local\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: overflow encountered in double_scalars


Time limit reached - bfgs
Time limit reached - gd
Time limit reached - polyak
Time limit reached - nesterov
Time limit reached - adagrad
Time limit reached - newton
Time limit reached - bfgs
Time limit reached - gd
Time limit reached - polyak
Time limit reached - nesterov
Time limit reached - adagrad
Time limit reached - newton
Time limit reached - bfgs
Distance from actual minimum with method GD:- at starting point [[1 1]]: 
    - after 0.1s: 2.0135193289455002
    - after 1s: 1.6067950624636331
    - after 2s: 1.341349813172955
- at starting point [[4.5 4.5]]: 
    - after 0.1s: 0.8120581424505792
    - after 1s: 0.7889390054797938
    - after 2s: 0.7824087473186083
Distance from actual minimum with method Polyak:- at starting point [[1 1]]: 
    - after 0.1s: 1.9722304427953623
    - after 1s: 1.3849181379892868
    - after 2s: 1.0837184376590137
- at starting point [[4.5 4.5]]: 
    - after 0.1s: 0.7715587911361718
    - after 1s: 0.7564268702572243
    - after 2s: 0.74501980558400

For our last function, the method of choice is Polyak GD.

### Problem 6

In [106]:
def prepare_data(N=100):
    random.seed(0)

    noise = np.array([random.random() for _ in range(N)])

    X = np.array(range(1,N+1)) / N
    Y = X + noise / N
    return X, Y

In [ ]:
def sgd_linear(pt1, lr, steps, X, Y):
    points = [pt1]
    for _ in range(steps):
        pt = points[-1]
        i = np.random.randint(0, len(X))
        x, y = X[i], Y[i]
        points.append(pt - lr*np.array([[x], [1]]) * 2*(pt[0]*x + pt[1] - y))
    return points

def l_bfgs(pt1, df, m, steps):
    points = [pt1, pt1 - df(pt1)]
    gammas = [df(points[0]) - df(points[1])]
    deltas = [df(pt1)]                          # pt1 - points[1]
    ros = [1/deltas[0].T.dot(gammas[0])]
    for s in range(steps):

        q = df(points[-1])
        if len(gammas) > m:
            gammas.pop(0)
            deltas.pop(0)
            ros.pop(0)
        alpha = np.zeros(len(deltas))
        for i in range(len(deltas)-1, -1, -1):
            alpha[i] = ros[i] * deltas[i].T.dot(q)
            q -= alpha[i] * gammas[i]
        r = (deltas[-1].T.dot(gammas[-1]) / gammas[-1].T.dot(gammas[-1])) * q
        for i in range(len(deltas)):
            beta = ros[i] * gammas[i].T.dot(r)
            r += deltas[i] * (alpha[i] - beta)

        prev_pt = points[-1]
        new_pt = prev_pt - r
        gamma = df(new_pt) - df(prev_pt)
        delta = new_pt - prev_pt
        ro = 1/gamma.T.dot(delta)

        points.append(new_pt)
        gammas.append(gamma)
        deltas.append(delta)
        ros.append(ro)

    return points


In [ ]:
N = 50
X, Y = prepare_data(N)

def f(k, n):
    return sum((k*X + n - Y)**2) / N

def gradf(par):
    k, n = par[0, 0], par[1, 0]
    return np.array([[2*(k*X + n - Y).dot(X), sum(2*(k*X + n - Y))]]).T / N

def Hf(par):
    return np.array([[2*sum(X**2)/ N, 2*sum(X)/ N], [2*sum(X)/ N, 2]])

t = time.time()
print(f'Gradient descent: {gd(np.array([[0],[1]]), gradf, 0.01, 5000)[-1].T} (5000 steps in {time.time()-t})')
t = time.time()
print(f'SGD: {sgd_linear(np.array([[0],[1]]), 0.01, 5000, X, Y)[-1].T} (5000 steps in {time.time()-t})')
t = time.time()
print(f'Newton: {newton(np.array([[0],[1]]), gradf, Hf, 200)[-1].T} (200 steps in {time.time()-t})')
t = time.time()
print(f'BFGS: {bfgs(np.array([[0],[1]]), gradf, 20)[-1].T} (20 steps in {time.time()-t})')
t = time.time()
print(f'L-BFGS: {l_bfgs(np.array([[0],[1]]), gradf, 5, 20)[-1].T} (20 steps in {time.time()-t})')

In [ ]:
N = 100
X, Y = prepare_data(N)

def f(k, n):
    return sum((k*X + n - Y)**2) / N

def gradf(par):
    k, n = par[0, 0], par[1, 0]
    return np.array([[2*(k*X + n - Y).dot(X), sum(2*(k*X + n - Y))]]).T / N

def Hf(par):
    return np.array([[2*sum(X**2)/ N, 2*sum(X)/ N], [2*sum(X)/ N, 2]])

t = time.time()
print(f'Gradient descent: {gd(np.array([[0],[1]]), gradf, 0.01, 5000)[-1].T} (5000 steps in {time.time()-t})')
t = time.time()
print(f'SGD: {sgd_linear(np.array([[0],[1]]), 0.01, 5000, X, Y)[-1].T} (5000 steps in {time.time()-t})')
t = time.time()
print(f'Newton: {newton(np.array([[0],[1]]), gradf, Hf, 200)[-1].T} (200 steps in {time.time()-t})')
t = time.time()
print(f'BFGS: {bfgs(np.array([[0],[1]]), gradf, 20)[-1].T} (20 steps in {time.time()-t})')
t = time.time()
print(f'L-BFGS: {l_bfgs(np.array([[0],[1]]), gradf, 5, 20)[-1].T} (20 steps in {time.time()-t})')

In [ ]:
N = 1000
X, Y = prepare_data(N)

def f(k, n):
    return sum((k*X + n - Y)**2) / N

def gradf(par):
    k, n = par[0, 0], par[1, 0]
    return np.array([[2*(k*X + n - Y).dot(X), sum(2*(k*X + n - Y))]]).T / N

def Hf(par):
    return np.array([[2*sum(X**2)/ N, 2*sum(X)/ N], [2*sum(X)/ N, 2]])

t = time.time()
print(f'Gradient descent: {gd(np.array([[0],[1]]), gradf, 0.01, 5000)[-1].T} (5000 steps in {time.time()-t})')
t = time.time()
print(f'SGD: {sgd_linear(np.array([[0],[1]]), 0.01, 5000, X, Y)[-1].T} (5000 steps in {time.time()-t})')
t = time.time()
print(f'Newton: {newton(np.array([[0],[1]]), gradf, Hf, 200)[-1].T} (200 steps in {time.time()-t})')
t = time.time()
print(f'BFGS: {bfgs(np.array([[0],[1]]), gradf, 20)[-1].T} (20 steps in {time.time()-t})')
t = time.time()
print(f'L-BFGS: {l_bfgs(np.array([[0],[1]]), gradf, 5, 20)[-1].T} (20 steps in {time.time()-t})')

In [ ]:
N = 10000
X, Y = prepare_data(N)

def f(k, n):
    return sum((k*X + n - Y)**2) / N

def gradf(par):
    k, n = par[0, 0], par[1, 0]
    return np.array([[2*(k*X + n - Y).dot(X), sum(2*(k*X + n - Y))]]).T / N

def Hf(par):
    return np.array([[2*sum(X**2)/ N, 2*sum(X)/ N], [2*sum(X)/ N, 2]])

t = time.time()
print(f'Gradient descent: {gd(np.array([[0],[1]]), gradf, 0.01, 1000)[-1].T} (1000 steps in {time.time()-t})')
t = time.time()
print(f'SGD: {sgd_linear(np.array([[0],[1]]), 0.01, 5000, X, Y)[-1].T} (5000 steps in {time.time()-t})')
t = time.time()
print(f'Newton: {newton(np.array([[0],[1]]), gradf, Hf, 100)[-1].T} (100 steps in {time.time()-t})')
t = time.time()
print(f'BFGS: {bfgs(np.array([[0],[1]]), gradf, 20)[-1].T} (20 steps in {time.time()-t})')
t = time.time()
print(f'L-BFGS: {l_bfgs(np.array([[0],[1]]), gradf, 5, 20)[-1].T} (20 steps in {time.time()-t})')

In [ ]:
N = 100000
X, Y = prepare_data(N)

def f(k, n):
    return sum((k*X + n - Y)**2) / N

def gradf(par):
    k, n = par[0, 0], par[1, 0]
    return np.array([[2*(k*X + n - Y).dot(X), sum(2*(k*X + n - Y))]]).T / N

def Hf(par):
    return np.array([[2*sum(X**2)/ N, 2*sum(X)/ N], [2*sum(X)/ N, 2]])

t = time.time()
print(f'Gradient descent: {gd(np.array([[0],[1]]), gradf, 0.01, 500)[-1].T} (500 steps in {time.time()-t})')
t = time.time()
print(f'SGD: {sgd_linear(np.array([[0],[1]]), 0.01, 100000, X, Y)[-1].T} (100000 steps in {time.time()-t})')
t = time.time()
print(f'Newton: {newton(np.array([[0],[1]]), gradf, Hf, 20)[-1].T} (20 steps in {time.time()-t})')
t = time.time()
print(f'BFGS: {bfgs(np.array([[0],[1]]), gradf, 20)[-1].T} (20 steps in {time.time()-t})')
t = time.time()
print(f'L-BFGS: {l_bfgs(np.array([[0],[1]]), gradf, 5, 20)[-1].T} (20 steps in {time.time()-t})')

In [ ]:
N = 1000000
X, Y = prepare_data(N)

def f(k, n):
    return sum((k*X + n - Y)**2) / N

def gradf(par):
    k, n = par[0, 0], par[1, 0]
    return np.array([[2*(k*X + n - Y).dot(X), sum(2*(k*X + n - Y))]]).T / N

def Hf(par):
    return np.array([[2*sum(X**2)/ N, 2*sum(X)/ N], [2*sum(X)/ N, 2]])

t = time.time()
print(f'Gradient descent: {gd(np.array([[0],[1]]), gradf, 0.01, 100)[-1].T} (100 steps in {time.time()-t})')
t = time.time()
print(f'SGD: {sgd_linear(np.array([[0],[1]]), 0.01, 1000000, X, Y)[-1].T} (1000000 steps in {time.time()-t})')
t = time.time()
print(f'Newton: {newton(np.array([[0],[1]]), gradf, Hf, 20)[-1].T} (20 steps in {time.time()-t})')
t = time.time()
print(f'BFGS: {bfgs(np.array([[0],[1]]), gradf, 20)[-1].T} (20 steps in {time.time()-t})')
t = time.time()
print(f'L-BFGS: {l_bfgs(np.array([[0],[1]]), gradf, 5, 20)[-1].T} (20 steps in {time.time()-t})')

We can see that second order methods are much better than GD, since they are faster and more precise, in less steps. GD fails in large datasets, since we would need too much time to perform enough steps for good results. But SGD has much faster steps, so it reaches similar performance as second order methods in the same time.

In [107]:
We can see that second order methods are much better than GD, since they are faster and more precise, in less steps. GD fails in large datasets, since we would need too much time to perform enough steps for good results. But SGD has much faster steps, so it reaches similar performance as second order methods in the same time.

In [117]:
N = 50
X, Y = prepare_data(N)

def f(k, n):
    return sum((k*X + n - Y)**2) / N

def gradf(par):
    k, n = par[0, 0], par[1, 0]
    return np.array([[2*(k*X + n - Y).dot(X), sum(2*(k*X + n - Y))]]).T / N

def Hf(par):
    return np.array([[2*sum(X**2)/ N, 2*sum(X)/ N], [2*sum(X)/ N, 2]])

t = time.time()
print(f'Gradient descent: {gd(np.array([[0],[1]]), gradf, 0.01, 5000)[-1].T} (5000 steps in {time.time()-t})')
t = time.time()
print(f'SGD: {sgd_linear(np.array([[0],[1]]), 0.01, 5000, X, Y)[-1].T} (5000 steps in {time.time()-t})')
t = time.time()
print(f'Newton: {newton(np.array([[0],[1]]), gradf, Hf, 200)[-1].T} (200 steps in {time.time()-t})')
t = time.time()
print(f'BFGS: {bfgs(np.array([[0],[1]]), gradf, 20)[-1].T} (20 steps in {time.time()-t})')
t = time.time()
print(f'L-BFGS: {l_bfgs(np.array([[0],[1]]), gradf, 5, 20)[-1].T} (20 steps in {time.time()-t})')

Gradient descent: [[0.99652904 0.01332757]] (5000 steps in 0.12879157066345215)
SGD: [[0.9968744  0.01258143]] (5000 steps in 0.06503891944885254)
Newton: [[0.99827278 0.01237627]] (200 steps in 0.013986349105834961)
bfgs: stopped because of invalid values
BFGS: [[0.99827278 0.01237627]] (20 steps in 0.0020110607147216797)
L-BFGS: [[0.99827278 0.01237627]] (20 steps in 0.0029997825622558594)


In [118]:
N = 100
X, Y = prepare_data(N)

def f(k, n):
    return sum((k*X + n - Y)**2) / N

def gradf(par):
    k, n = par[0, 0], par[1, 0]
    return np.array([[2*(k*X + n - Y).dot(X), sum(2*(k*X + n - Y))]]).T / N

def Hf(par):
    return np.array([[2*sum(X**2)/ N, 2*sum(X)/ N], [2*sum(X)/ N, 2]])

t = time.time()
print(f'Gradient descent: {gd(np.array([[0],[1]]), gradf, 0.01, 5000)[-1].T} (5000 steps in {time.time()-t})')
t = time.time()
print(f'SGD: {sgd_linear(np.array([[0],[1]]), 0.01, 5000, X, Y)[-1].T} (5000 steps in {time.time()-t})')
t = time.time()
print(f'Newton: {newton(np.array([[0],[1]]), gradf, Hf, 200)[-1].T} (200 steps in {time.time()-t})')
t = time.time()
print(f'BFGS: {bfgs(np.array([[0],[1]]), gradf, 20)[-1].T} (20 steps in {time.time()-t})')
t = time.time()
print(f'L-BFGS: {l_bfgs(np.array([[0],[1]]), gradf, 5, 20)[-1].T} (20 steps in {time.time()-t})')

Gradient descent: [[0.99890635 0.00646922]] (5000 steps in 0.1719987392425537)
SGD: [[0.99837604 0.0067175 ]] (5000 steps in 0.06699919700622559)
Newton: [[1.00060802 0.00554968]] (200 steps in 0.019998788833618164)
BFGS: [[1.00060802 0.00554968]] (20 steps in 0.003000497817993164)
L-BFGS: [[1.00060802 0.00554968]] (20 steps in 0.00400090217590332)


In [119]:
N = 1000
X, Y = prepare_data(N)

def f(k, n):
    return sum((k*X + n - Y)**2) / N

def gradf(par):
    k, n = par[0, 0], par[1, 0]
    return np.array([[2*(k*X + n - Y).dot(X), sum(2*(k*X + n - Y))]]).T / N

def Hf(par):
    return np.array([[2*sum(X**2)/ N, 2*sum(X)/ N], [2*sum(X)/ N, 2]])

t = time.time()
print(f'Gradient descent: {gd(np.array([[0],[1]]), gradf, 0.01, 5000)[-1].T} (5000 steps in {time.time()-t})')
t = time.time()
print(f'SGD: {sgd_linear(np.array([[0],[1]]), 0.01, 5000, X, Y)[-1].T} (5000 steps in {time.time()-t})')
t = time.time()
print(f'Newton: {newton(np.array([[0],[1]]), gradf, Hf, 200)[-1].T} (200 steps in {time.time()-t})')
t = time.time()
print(f'BFGS: {bfgs(np.array([[0],[1]]), gradf, 20)[-1].T} (20 steps in {time.time()-t})')
t = time.time()
print(f'L-BFGS: {l_bfgs(np.array([[0],[1]]), gradf, 5, 20)[-1].T} (20 steps in {time.time()-t})')

Gradient descent: [[0.99829198 0.00140977]] (5000 steps in 0.8229990005493164)
SGD: [[0.99846352 0.00127669]] (5000 steps in 0.0691986083984375)
Newton: [[9.99954789e-01 5.18998970e-04]] (200 steps in 0.12718534469604492)
BFGS: [[9.99954789e-01 5.18998970e-04]] (20 steps in 0.011997222900390625)
L-BFGS: [[9.99954789e-01 5.18998970e-04]] (20 steps in 0.01399993896484375)


In [120]:
N = 10000
X, Y = prepare_data(N)

def f(k, n):
    return sum((k*X + n - Y)**2) / N

def gradf(par):
    k, n = par[0, 0], par[1, 0]
    return np.array([[2*(k*X + n - Y).dot(X), sum(2*(k*X + n - Y))]]).T / N

def Hf(par):
    return np.array([[2*sum(X**2)/ N, 2*sum(X)/ N], [2*sum(X)/ N, 2]])

t = time.time()
print(f'Gradient descent: {gd(np.array([[0],[1]]), gradf, 0.01, 1000)[-1].T} (1000 steps in {time.time()-t})')
t = time.time()
print(f'SGD: {sgd_linear(np.array([[0],[1]]), 0.01, 5000, X, Y)[-1].T} (5000 steps in {time.time()-t})')
t = time.time()
print(f'Newton: {newton(np.array([[0],[1]]), gradf, Hf, 100)[-1].T} (100 steps in {time.time()-t})')
t = time.time()
print(f'BFGS: {bfgs(np.array([[0],[1]]), gradf, 20)[-1].T} (20 steps in {time.time()-t})')
t = time.time()
print(f'L-BFGS: {l_bfgs(np.array([[0],[1]]), gradf, 5, 20)[-1].T} (20 steps in {time.time()-t})')

Gradient descent: [[0.67982156 0.17142094]] (1000 steps in 2.111002206802368)
SGD: [[9.98438178e-01 9.06125523e-04]] (5000 steps in 0.08395934104919434)
Newton: [[9.99998330e-01 5.09011207e-05]] (100 steps in 0.7059998512268066)
BFGS: [[9.99998330e-01 5.09011207e-05]] (20 steps in 0.1100010871887207)
L-BFGS: [[9.99998330e-01 5.09011207e-05]] (20 steps in 0.10381293296813965)


In [121]:
N = 100000
X, Y = prepare_data(N)

def f(k, n):
    return sum((k*X + n - Y)**2) / N

def gradf(par):
    k, n = par[0, 0], par[1, 0]
    return np.array([[2*(k*X + n - Y).dot(X), sum(2*(k*X + n - Y))]]).T / N

def Hf(par):
    return np.array([[2*sum(X**2)/ N, 2*sum(X)/ N], [2*sum(X)/ N, 2]])

t = time.time()
print(f'Gradient descent: {gd(np.array([[0],[1]]), gradf, 0.01, 500)[-1].T} (500 steps in {time.time()-t})')
t = time.time()
print(f'SGD: {sgd_linear(np.array([[0],[1]]), 0.01, 100000, X, Y)[-1].T} (100000 steps in {time.time()-t})')
t = time.time()
print(f'Newton: {newton(np.array([[0],[1]]), gradf, Hf, 20)[-1].T} (20 steps in {time.time()-t})')
t = time.time()
print(f'BFGS: {bfgs(np.array([[0],[1]]), gradf, 20)[-1].T} (20 steps in {time.time()-t})')
t = time.time()
print(f'L-BFGS: {l_bfgs(np.array([[0],[1]]), gradf, 5, 20)[-1].T} (20 steps in {time.time()-t})')

Gradient descent: [[0.38187357 0.33082066]] (500 steps in 9.945998668670654)
SGD: [[9.99999657e-01 5.26353505e-06]] (100000 steps in 1.4070022106170654)
Newton: [[1.00000002e+00 4.98516617e-06]] (20 steps in 1.568998098373413)
BFGS: [[1.00000002e+00 4.98516617e-06]] (20 steps in 1.1319999694824219)
L-BFGS: [[1.00000002e+00 4.98516617e-06]] (20 steps in 1.296036958694458)


In [122]:
N = 1000000
X, Y = prepare_data(N)

def f(k, n):
    return sum((k*X + n - Y)**2) / N

def gradf(par):
    k, n = par[0, 0], par[1, 0]
    return np.array([[2*(k*X + n - Y).dot(X), sum(2*(k*X + n - Y))]]).T / N

def Hf(par):
    return np.array([[2*sum(X**2)/ N, 2*sum(X)/ N], [2*sum(X)/ N, 2]])

t = time.time()
print(f'Gradient descent: {gd(np.array([[0],[1]]), gradf, 0.01, 100)[-1].T} (100 steps in {time.time()-t})')
t = time.time()
print(f'SGD: {sgd_linear(np.array([[0],[1]]), 0.01, 1000000, X, Y)[-1].T} (1000000 steps in {time.time()-t})')
t = time.time()
print(f'Newton: {newton(np.array([[0],[1]]), gradf, Hf, 20)[-1].T} (20 steps in {time.time()-t})')
t = time.time()
print(f'BFGS: {bfgs(np.array([[0],[1]]), gradf, 20)[-1].T} (20 steps in {time.time()-t})')
t = time.time()
print(f'L-BFGS: {l_bfgs(np.array([[0],[1]]), gradf, 5, 20)[-1].T} (20 steps in {time.time()-t})')

Gradient descent: [[-0.03142227  0.58765207]] (100 steps in 24.45909595489502)
SGD: [[1.00000007e+00 5.46574430e-07]] (1000000 steps in 14.115238189697266)
Newton: [[9.99999999e-01 5.00281263e-07]] (20 steps in 13.457749605178833)
BFGS: [[9.99999999e-01 5.00281263e-07]] (20 steps in 13.58800745010376)
L-BFGS: [[9.99999999e-01 5.00281263e-07]] (20 steps in 15.257956504821777)


We can see that second order methods are much better than GD, since they are faster and more precise, in less steps. GD fails in large datasets, since we would need too much time to perform enough steps for good results. But SGD has much faster steps, so it reaches similar performance as second order methods in the same time.